# WORK Token Distribution

A cadCAD sketch for the Workbench WORK token distribution.


## Check cadCAD

This cell doesn't do anything, but it does ensure that you know what version of cadCAD you're running. That way if/when you encounter technical difficulties you can tell the community which version of cadCAD you're running. Might save you hours of pain if all you need to do is upgrade to the latest version. 

In [70]:
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /usr/local/lib/python3.8/site-packages
Requires: pandas, pathos, fn, funcy
Required-by: cadCAD-diagram
Note: you may need to restart the kernel to use updated packages.


## Import Stuff

These are the libraries you'll need (cadCAD stuff) and that could be useful (python stuff) for your cadCAD model.

In [71]:
# python stuff
import numpy as np
import random as random

# cadCAD stuff
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

## Initial Params

These are the parameters that control the behavior of a system. For example, if you were modeling a Web3 protocol these would be the parameters within a system that token holders could vote to modify. All of the parameters of a protocol that can be modified should be accessible here. That way it's easy to modify and run similations to explore how parameter changes might affect the network.

In [72]:
# the initial state of the cadCAD model
# in a stock and flow model you can think of this as the stocks
genesis_states = {
    'tokens': 1,
    'weekly_token_minting': 100,
    'community_treasury': 0,
    'contributors': {
        'token_balance': 0
    }
}

# The parameters to run the model
# Often these are found towards the end of the file near the cadCAD engine
# but we're putting them here so that you can easily configure and run 
# the notebook without having to scroll back and forth
sim_config_dict = {
    # timesteps: weekly
    'T': range(52), 
    # parallel runs
    'N': 3,
    # more advanced stuff
    #'M': {}
}

## Policy Functions

Policy functions are like flows in stock and flow diagrams. They modify the inputs to state update functions.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks. 

In [73]:
# Mint tokens on a weekly basis (each timestep)
def p_mint_tokens(params, step, sH, s):
    minted_tokens = s['weekly_token_minting']
    return ({'minted_tokens': minted_tokens})

# Do work to earn tokens
def p_do_work(params, step, sH, s):
    # some contributions create more value than others
    impact = random.randint(1, 10)
    return ({'impact': impact})

## State Update Functions

These functions take in inputs (state variables and policies) and modify the state.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks.

In [74]:
# Update token count based on weekly minting
def s_update_token_count(params, step, sH, s, _input):
    y = 'tokens'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Put newly minted tokens into the community treasury
def s_update_community_treasury(params, step, sH, s, _input):
    y = 'community_treasury'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Recognize and reward community contributors
def s_reward_contributions(params, step, sH, s, _input):
    y = 'contributors'
    x = s[y]
    x['token_balance'] += _input['impact']
    return (y, x)


## State Update Block

If you're approaching cadCAD from Web3 you can think of the state of the model as something that evolves in blocks. Each block has a set of actions that updates the state. Those transactions then get batched into blocks to be processed together. In cadCAD blocks are called "`partial_state_update_blocks`." As you can see below, this is an array that is very similar to a "block" in a blockchain in that it represents a set of actions to update the state. That state is then updated across many timesteps. This is similar to how the state of a blockchain is updated over many timesteps as new blocks are added.



In [75]:
partial_state_update_blocks = [
    { 
        # mint tokens
        'policies': { 
            'minted_tokens': p_mint_tokens
        },
        # update token count and community treasury
        'variables': {
            'tokens': s_update_token_count,
            'community_treasury': s_update_community_treasury
        }
    },
    {
        # do work
        'policies': {
            'impact': p_do_work
        },
        # recognize and reward contributors
        'variables': {
            'contributors': s_reward_contributions
        }
    }
]

## Running the cadCAD Engine

In [76]:
# imported some addition utilities to help with configuration set-up
exp = Experiment()
c = config_sim(sim_config_dict)
# The configurations above are then packaged into a `Configuration` object
del configs[:]
# dict containing variable names and initial values
exp.append_configs(initial_state=genesis_states, 
                    # dict containing state update functions 
                    partial_state_update_blocks=partial_state_update_blocks, 
                    # preprocessed dictionaries containing simulation parameters
                    sim_configs=c)


In [77]:
%%capture
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(exec_mode.local_mode)

# pass the configuration object inside an array
simulation = Executor(exec_context=local_mode_ctx, configs=configs) 
# the `execute()` method returns a tuple; its first elements contains the raw results
raw_system_events, tensor_field, sessions = simulation.execute() 

## Data Visualization

This is often half the battle. Not only do need to design and build a cadCAD model, but you need to understand how it's working and be able to effectively communicate that to other people. A picture says a thousand words, thus enter data viz. Getting good at using python data viz libraries is probably the highest leverage thing you can do after you learn the cadCAD basics.

In [78]:
%matplotlib inline
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

tokens  weekly_token_minting  community_treasury  \
subset run timestep substep                                                     
0      1   0        0             1                   100                   0   
           1        1           101                   100                 100   
                    2           101                   100                 100   
           2        1           201                   100                 200   
                    2           201                   100                 200   
...                             ...                   ...                 ...   
       3   50       2          5001                   100                5000   
           51       1          5101                   100                5100   
                    2          5101                   100                5100   
           52       1          5201                   100                5200   
                    2          5201                   100                5200   

                                       contributors  simulation  
subset run timestep substep                                      
0      1   0        0          {'token_balance': 0}           0  
           1        1          {'token_balance': 0}           0  
                    2          {'token_balance': 3}           0  
           2        1          {'token_balance': 3}           0  
                    2          {'token_balance': 4}           0  
...                                             ...         ...  
       3   50       2        {'token_balance': 283}           0  
           51       1        {'token_balance': 283}           0  
                    2        {'token_balance': 286}           0  
           52       1        {'token_balance': 286}           0  
                    2        {'token_balance': 295}           0  

[315 rows x 5 columns]

simulation_result.plot('timestep', ['tokens', 'community_treasury'], grid=True, 
        colormap = 'gist_rainbow',
        xticks=list(simulation_result['timestep'].drop_duplicates()), 
        yticks=list(range(1+(simulation_result['tokens']+simulation_result['community_treasury']).max())))